In [2]:
from glob import glob
import os 
from uvars import gedi_grid_dpath

In [13]:
files = glob(f'{gedi_grid_dpath}/*/*90.tif', recursive=True); print(len(files))
#files = [i for i in ]

68


In [15]:
dtm_files = [i for i in files if 'dtmT' in i];print(len(dtm_files))
dsm_files = [i for i in files if 'dsmT' in i];print(len(dsm_files))

17
17


In [19]:
import numpy as np
import rasterio
from rasterio.transform import from_origin

def gen_label_by_man_threshold(dsm_path, dtm_path, mask_path):
    # Abrir os arquivos DSM e DTM
    with rasterio.open(dsm_path) as dsm_ds, rasterio.open(dtm_path) as dtm_ds:
        dsm_data = dsm_ds.read(1)
        dtm_data = dtm_ds.read(1)
        transform = dsm_ds.transform
        crs = dsm_ds.crs

    # Substituir valores NoData por NaN
    dsm_data = np.where(dsm_data == dsm_ds.nodata, np.nan, dsm_data)
    dtm_data = np.where(dtm_data == dtm_ds.nodata, np.nan, dtm_data)

    # Filtrar valores extremos
    dsm_data[(dsm_data < -999) | (dsm_data > 1000)] = np.nan
    dtm_data[(dtm_data < -999) | (dtm_data > 1000)] = np.nan

    # Garantir que as dimensões são iguais
    if dsm_data.shape != dtm_data.shape:
        raise ValueError("DSM e DTM devem ter as mesmas dimensões.")

    # Calcular a diferença entre DSM e DTM
    diff_data = dsm_data - dtm_data
    
    # Determinar o limiar automaticamente (exemplo: percentil 90%)
    valid_diff = diff_data[~np.isnan(diff_data)]
    threshold = np.percentile(valid_diff, 90)
    
    # Atualizar o nome do arquivo
    threshold_str = str(round(threshold, 2)).replace('.', 'p')
    mask_path = mask_path.replace('.tif', f'_{threshold_str}.tif')

    # Criar a máscara
    mask = np.full(dsm_data.shape, 0, dtype=np.uint8)
    mask[np.isnan(dsm_data) | np.isnan(dtm_data)] = 2
    mask[(~np.isnan(dsm_data)) & (~np.isnan(dtm_data)) & (diff_data < threshold)] = 1

    # Criar o arquivo de saída
    with rasterio.open(
        mask_path, 'w', driver='GTiff',
        height=mask.shape[0], width=mask.shape[1],
        count=1, dtype=rasterio.uint8,
        crs=crs, transform=transform
    ) as mask_ds:
        mask_ds.write(mask, 1)

    print(f"Máscara criada e salva em {mask_path} com threshold {threshold:.2f}")


In [23]:
import numpy as np
import rasterio
from scipy.ndimage import generic_filter

def local_threshold(array, size=5, percentile=90):
    """Calcula um limiar adaptativo com base em uma janela deslizante."""
    def func(window):
        return np.nanpercentile(window, percentile)
    
    return generic_filter(array, func, size=size, mode='nearest')

def gen_label_by_man_threshold(dsm_path, dtm_path, mask_path):
    # Abrir os arquivos DSM e DTM
    with rasterio.open(dsm_path) as dsm_ds, rasterio.open(dtm_path) as dtm_ds:
        dsm_data = dsm_ds.read(1)
        dtm_data = dtm_ds.read(1)
        transform = dsm_ds.transform
        crs = dsm_ds.crs

    # Substituir valores NoData por NaN
    dsm_data = np.where(dsm_data == dsm_ds.nodata, np.nan, dsm_data)
    dtm_data = np.where(dtm_data == dtm_ds.nodata, np.nan, dtm_data)

    # Filtrar valores extremos
    dsm_data[(dsm_data < -999) | (dsm_data > 1000)] = np.nan
    dtm_data[(dtm_data < -999) | (dtm_data > 1000)] = np.nan

    # Garantir que as dimensões são iguais
    if dsm_data.shape != dtm_data.shape:
        raise ValueError("DSM e DTM devem ter as mesmas dimensões.")

    # Calcular a diferença entre DSM e DTM
    diff_data = dsm_data - dtm_data
    
    # Calcular limiar adaptativo local
    local_thresholds = local_threshold(diff_data, size=7, percentile=90)
    
    # Atualizar o nome do arquivo
    threshold_str = "local"
    mask_path = mask_path.replace('.tif', f'_{threshold_str}.tif')

    # Criar a máscara
    mask = np.full(dsm_data.shape, 0, dtype=np.uint8)
    mask[np.isnan(dsm_data) | np.isnan(dtm_data)] = 2
    mask[(~np.isnan(dsm_data)) & (~np.isnan(dtm_data)) & (diff_data < local_thresholds)] = 1

    # Criar o arquivo de saída
    with rasterio.open(
        mask_path, 'w', driver='GTiff',
        height=mask.shape[0], width=mask.shape[1],
        count=1, dtype=rasterio.uint8,
        crs=crs, transform=transform
    ) as mask_ds:
        mask_ds.write(mask, 1)

    print(f"Máscara criada e salva em {mask_path} usando limiar adaptativo local.")


In [24]:
for dsm, dtm in zip(dtm_files, dsm_files):
    print(dsm,dtm)
    mask = dsm.replace('.tif', '_LABEL_LOCAL.tif')
    gen_label_by_man_threshold(dsm_path=dsm, dtm_path=dtm, mask_path=mask) # water should be dtm as zero only two values

/home/ljp238/Downloads/GEDI_L3_BULK/tiles/N13E103/N13E103_GEDI03_dtmT___GWR_90.tif /home/ljp238/Downloads/GEDI_L3_BULK/tiles/N13E103/N13E103_GEDI03_dsmT___GWR_90.tif
Máscara criada e salva em /home/ljp238/Downloads/GEDI_L3_BULK/tiles/N13E103/N13E103_GEDI03_dtmT___GWR_90_LABEL_LOCAL_local.tif usando limiar adaptativo local.
/home/ljp238/Downloads/GEDI_L3_BULK/tiles/S02W063/S02W063_GEDI03_dtmT___GWR_90.tif /home/ljp238/Downloads/GEDI_L3_BULK/tiles/S02W063/S02W063_GEDI03_dsmT___GWR_90.tif


/home/ljp238/miniconda3/envs/demvfill/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1384: RuntimeWarning: All-NaN slice encountered
  return _nanquantile_unchecked(


Máscara criada e salva em /home/ljp238/Downloads/GEDI_L3_BULK/tiles/S02W063/S02W063_GEDI03_dtmT___GWR_90_LABEL_LOCAL_local.tif usando limiar adaptativo local.
/home/ljp238/Downloads/GEDI_L3_BULK/tiles/N11E105/N11E105_GEDI03_dtmT___GWR_90.tif /home/ljp238/Downloads/GEDI_L3_BULK/tiles/N11E105/N11E105_GEDI03_dsmT___GWR_90.tif
Máscara criada e salva em /home/ljp238/Downloads/GEDI_L3_BULK/tiles/N11E105/N11E105_GEDI03_dtmT___GWR_90_LABEL_LOCAL_local.tif usando limiar adaptativo local.
/home/ljp238/Downloads/GEDI_L3_BULK/tiles/S02W064/S02W064_GEDI03_dtmT___GWR_90.tif /home/ljp238/Downloads/GEDI_L3_BULK/tiles/S02W064/S02W064_GEDI03_dsmT___GWR_90.tif
Máscara criada e salva em /home/ljp238/Downloads/GEDI_L3_BULK/tiles/S02W064/S02W064_GEDI03_dtmT___GWR_90_LABEL_LOCAL_local.tif usando limiar adaptativo local.
/home/ljp238/Downloads/GEDI_L3_BULK/tiles/N12E103/N12E103_GEDI03_dtmT___GWR_90.tif /home/ljp238/Downloads/GEDI_L3_BULK/tiles/N12E103/N12E103_GEDI03_dsmT___GWR_90.tif
Máscara criada e salva em

In [17]:
import numpy as np
import rasterio

def create_labels_by_compare(dtm_path, dsm_path, output_path):
    """
    Process two DEM rasters (DTM and DSM) and create a new raster based on specified conditions.

    Args:
        dtm_path (str): Path to the Digital Terrain Model (DTM) raster.
        dsm_path (str): Path to the Digital Surface Model (DSM) raster.
        output_path (str): Path to save the output raster.

    Returns:
        None
    """
    # Open DEM rasters
    with rasterio.open(dtm_path) as dtm_src, rasterio.open(dsm_path) as dsm_src:
        # Read data as numpy arrays
        dtm = dtm_src.read(1, masked=True)  # Read as masked array to handle nodata
        dsm = dsm_src.read(1, masked=True)
        
        # Initialize an output array with the same shape as the DEMs
        result = np.full(dtm.shape, 2, dtype=np.uint8)  # Default to 2 for both nulls
        
        # Assign values based on conditions
        result[(~dtm.mask) & (dsm.mask)] = 1  # dtm valid, dsm null
        result[(dtm.mask) & (~dsm.mask)] = 0  # dtm null, dsm valid
        result[(~dtm.mask) & (~dsm.mask)] = 1  # Both valid, prioritize dtm
        
        # Write the result to a new raster
        with rasterio.open(
            output_path,
            'w',
            driver='GTiff',
            height=result.shape[0],
            width=result.shape[1],
            count=1,
            dtype=result.dtype,
            crs=dtm_src.crs,
            transform=dtm_src.transform,
        ) as dst:
            dst.write(result, 1)

    print(f"Output raster saved to {output_path}")



In [18]:
def gen_label_by_man_threshold(dsm_path, dtm_path, mask_path, threshold=0.5):
    #import numpy as np
    #from osgeo import gdal

    threshold = str(threshold).replace('.', 'p')
    mask_path = mask_path.replace('.tif', f'_{threshold}.tif')

    # Open the DSM and DTM files
    dsm_ds = gdal.Open(dsm_path)
    dtm_ds = gdal.Open(dtm_path)

    if dsm_ds is None or dtm_ds is None:
        raise FileNotFoundError("DSM or DTM file not found.")

    # Read the first band (assuming single-band rasters)
    dsm_band = dsm_ds.GetRasterBand(1)
    dtm_band = dtm_ds.GetRasterBand(1)

    dsm_data = dsm_band.ReadAsArray()
    dtm_data = dtm_band.ReadAsArray()

    # Get NoData values from the bands
    dsm_nodata = dsm_band.GetNoDataValue()
    dtm_nodata = dtm_band.GetNoDataValue()

    # Set NoData values to np.nan
    if dsm_nodata is not None:
        dsm_data[dsm_data == dsm_nodata] = np.nan
    if dtm_nodata is not None:
        dtm_data[dtm_data == dtm_nodata] = np.nan

    # Filter values < -999 and > 1000 and set to np.nan
    dsm_data[(dsm_data < -999) | (dsm_data > 1000)] = np.nan
    dtm_data[(dtm_data < -999) | (dtm_data > 1000)] = np.nan

    # Ensure both arrays have the same shape
    if dsm_data.shape != dtm_data.shape:
        raise ValueError("DSM and DTM must have the same dimensions.")

    # Create the mask array based on the criteria
    mask = np.full(dsm_data.shape, 0, dtype=np.uint8)
    mask[np.isnan(dsm_data) | np.isnan(dtm_data)] = 2
    mask[(~np.isnan(dsm_data)) & (~np.isnan(dtm_data)) & ((dsm_data - dtm_data) < threshold)] = 1

    # Create the output mask file
    driver = gdal.GetDriverByName('GTiff')
    mask_ds = driver.Create(mask_path, dsm_ds.RasterXSize, dsm_ds.RasterYSize, 1, gdal.GDT_Byte)

    # Set the same georeference as the input files
    mask_ds.SetGeoTransform(dsm_ds.GetGeoTransform())
    mask_ds.SetProjection(dsm_ds.GetProjection())

    # Write the mask array to the output file
    mask_band = mask_ds.GetRasterBand(1)
    mask_band.WriteArray(mask)

    # Close the datasets
    dsm_ds = None
    dtm_ds = None
    mask_ds = None

    print(f"Mask created and saved to {mask_path}")